<a href="https://colab.research.google.com/github/xi1224-pan/ml-1/blob/main/%E8%AE%8A%E5%BD%A2%E9%87%91%E5%89%9B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers


In [2]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

model_name = "uer/roberta-base-finetuned-jd-binary-chinese"

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/409M [00:00<?, ?B/s]

In [3]:
def predict_sentiment(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=64)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        label = torch.argmax(probs).item()
    return "正面" if label == 1 else "負面"



In [4]:
print(predict_sentiment("烤大腸很讚，味道都還不錯牛肉也嫩，還有少見的大阪燒，上菜很快技術好"))
print(predict_sentiment("料理，串燒，非常道地這家店"))
print(predict_sentiment("幹你娘"))


model.safetensors:   0%|          | 0.00/409M [00:00<?, ?B/s]

正面
正面
負面


In [8]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# 1. 載入 Hugging Face 線上模型
model_name = "uer/roberta-base-finetuned-jd-binary-chinese"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# 2. 定義情緒分析函數（回傳情緒 + 分數）
def predict_sentiment_with_score(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=64)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        score = probs[0][1].item() - probs[0][0].item()  # 正面機率 - 負面機率

    # 分類規則：你可以根據需求調整
    if score > 1:
        sentiment = "強烈正面"
    elif score > 0:
        sentiment = "正面"
    elif score == 0:
        sentiment = "中立"
    else:
        sentiment = "負面"

    return {
        "情緒分類": sentiment,
        "情緒分數": round(score, 4),

    }

# 3. 測試範例
print(predict_sentiment_with_score("住附近飯店，散步過來用餐，環境不錯，小哥服務也好"))
print(predict_sentiment_with_score("股市下跌"))
print(predict_sentiment_with_score("AI 題材持續發酵，相關個股大漲"))


{'情緒分類': '正面', '情緒分數': 0.9875}
{'情緒分類': '負面', '情緒分數': -0.1469}
{'情緒分類': '正面', '情緒分數': 0.9489}
